#Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
!pip install kaggle

# Carga de los datos

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 2.07GB/s]


In [ ]:
!unzip -l udea-ai-4-eng-20251-pruebas-saber-pro-colombia

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
  4716673  2025-03-25 16:54   submission_example.csv
 59185250  2025-03-25 16:54   test.csv
143732449  2025-03-25 16:54   train.csv
---------                     -------
207634372                     3 files


In [ ]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!ls -lash *.csv

4.5M -rw-r--r-- 1 root root 4.5M Mar 25 16:54 submission_example.csv
 57M -rw-r--r-- 1 root root  57M Mar 25 16:54 test.csv
138M -rw-r--r-- 1 root root 138M Mar 25 16:54 train.csv


# Exploración

In [ ]:
dtr = pd.read_csv('train.csv')

In [ ]:
dtr.shape

(692500, 21)

In [ ]:
dtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object 
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object 
 7   FAMI_TIENEINTERNET              665871 non-null  object 
 8   FAMI_EDUCACIONPADRE             669322 non-null  object 
 9   FAMI_TIENELAVADORA              652727 non-null  object 
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object 
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 12  ESTU_PAGOMATRICU

In [ ]:
dtr.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
for i in dtr.columns:
    print ("%20s"%i, np.sum(dtr[i].isna()))

                  ID 0
             PERIODO 0
 ESTU_PRGM_ACADEMICO 0
ESTU_PRGM_DEPARTAMENTO 0
ESTU_VALORMATRICULAUNIVERSIDAD 6287
ESTU_HORASSEMANATRABAJA 30857
FAMI_ESTRATOVIVIENDA 32137
  FAMI_TIENEINTERNET 26629
 FAMI_EDUCACIONPADRE 23178
  FAMI_TIENELAVADORA 39773
 FAMI_TIENEAUTOMOVIL 43623
ESTU_PRIVADO_LIBERTAD 0
ESTU_PAGOMATRICULAPROPIO 6498
FAMI_TIENECOMPUTADOR 38103
FAMI_TIENEINTERNET.1 26629
 FAMI_EDUCACIONMADRE 23664
  RENDIMIENTO_GLOBAL 0
              coef_1 0
              coef_2 0
              coef_3 0
              coef_4 0


In [ ]:
dtr.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [ ]:
dtr.RENDIMIENTO_GLOBAL.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


In [ ]:
dtr.ESTU_PRIVADO_LIBERTAD.value_counts()

,count
ESTU_PRIVADO_LIBERTAD,
N,692466
S,34


# Preprocesado

In [ ]:
#Eliminar columnas no informativas o poco relevantes
del(dtr['PERIODO'])
del(dtr['FAMI_TIENEINTERNET.1'])
del(dtr['FAMI_TIENELAVADORA'])
del(dtr['FAMI_TIENEAUTOMOVIL'])
del(dtr['ESTU_PRIVADO_LIBERTAD'])

In [ ]:
dtr.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
for i in dtr.columns:
    print ("%20s"%i, np.sum(dtr[i].isna()))

                  ID 0
 ESTU_PRGM_ACADEMICO 0
ESTU_PRGM_DEPARTAMENTO 0
ESTU_VALORMATRICULAUNIVERSIDAD 6287
ESTU_HORASSEMANATRABAJA 30857
FAMI_ESTRATOVIVIENDA 32137
  FAMI_TIENEINTERNET 26629
 FAMI_EDUCACIONPADRE 23178
ESTU_PAGOMATRICULAPROPIO 6498
FAMI_TIENECOMPUTADOR 38103
 FAMI_EDUCACIONMADRE 23664
  RENDIMIENTO_GLOBAL 0
              coef_1 0
              coef_2 0
              coef_3 0
              coef_4 0


## Completar datos faltantes

In [ ]:
# Como nuestras variables numéricas, no tienen datos faltantes
# Completamos datos en variables categóricas
cat_cols = dtr.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    dtr[col] = dtr[col].fillna('Desconocido')

## Separación de X y Y

In [ ]:
X = dtr.drop(columns=['RENDIMIENTO_GLOBAL'])
y = dtr['RENDIMIENTO_GLOBAL']

## Codificar variables categóricas

In [ ]:
dtr.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,19528
Desconocido,6287


In [ ]:
dtr.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191
Desconocido,30857


In [ ]:
dtr.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Desconocido,32137
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [ ]:
dtr.RENDIMIENTO_GLOBAL.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


In [ ]:
def codificarCategoricas(X):
  X = X.copy()
  encoders = {}

  one_hot_cols = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']
  label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']
  ordinal_cols = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA']

  ordinal_map = {
        'ESTU_VALORMATRICULAUNIVERSIDAD': [
            'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
            'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
            'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
            'Más de 7 millones', 'Desconocido'
        ],
        'ESTU_HORASSEMANATRABAJA': [
            '0', 'Menos de 10 horas', 'Entre 11 y 20 horas',
            'Entre 21 y 30 horas', 'Más de 30 horas', 'Desconocido'
        ],
        'FAMI_ESTRATOVIVIENDA': [
            'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3',
            'Estrato 4', 'Estrato 5', 'Estrato 6', 'Desconocido'
        ],
  }

  #Ordinal Encoder
  for col in ordinal_cols:
    if col in X.columns:
      orden = ordinal_map[col]
      oe = OrdinalEncoder(categories=[orden])
      X[[col]] = oe.fit_transform(X[[col]])
      encoders[col] = oe

  #One-hot
  X = pd.get_dummies(X, columns=one_hot_cols, drop_first=False)

  #Label Encoder
  for col in label_cols:
    if col in X.columns:
      le = LabelEncoder()
      X[col] = X[col].fillna("__desconocido__")
      X[col] = X[col].astype(str)

      # Agregar "__desconocido__" como clase si no está
      if "__desconocido__" not in X[col].unique():
        X[col] = X[col].astype(str).replace(np.nan, "__desconocido__")

      le.fit(np.append(X[col].unique(), "__desconocido__"))
      X[col] = le.transform(X[col])
      encoders[col] = le

  # Guardar columnas para el test
  encoders['feature_names'] = X.columns.tolist()

  return X, encoders

In [ ]:
Xtr, encoders = codificarCategoricas(X)

In [ ]:
Xtr.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,coef_1,coef_2,...,coef_4,FAMI_TIENEINTERNET_Desconocido,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Desconocido,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Desconocido,FAMI_TIENECOMPUTADOR_No,FAMI_TIENECOMPUTADOR_Si
0,904256,300,4,6.0,1.0,3.0,12,6,0.322,0.208,...,0.267,False,False,True,False,True,False,False,False,True
1,645256,249,3,4.0,0.0,3.0,11,12,0.311,0.215,...,0.264,False,True,False,False,True,False,False,False,True
2,308367,819,4,4.0,4.0,3.0,9,9,0.297,0.214,...,0.264,False,False,True,False,True,False,False,True,False
3,470353,14,26,5.0,0.0,4.0,5,9,0.485,0.172,...,0.190,False,False,True,False,True,False,False,False,True
4,989032,907,1,4.0,3.0,3.0,7,7,0.316,0.232,...,0.294,False,False,True,False,True,False,False,False,True


**Ahora, se usa Ordinal Encoder para la variable de rendimiento que es nuestro *target***

In [ ]:
oe_y = OrdinalEncoder(categories=[['bajo', 'medio-bajo', 'medio-alto', 'alto']])
ytr = oe_y.fit_transform(y.values.reshape(-1, 1)).ravel()

In [ ]:
pd.DataFrame({
    "original": y,
    "codificado": ytr
}).head()

,original,codificado
0,medio-alto,2.0
1,bajo,0.0
2,bajo,0.0
3,alto,3.0
4,medio-bajo,1.0


## Carga y Preprocesado del archivo test

In [ ]:
dts = pd.read_csv('test.csv')

In [ ]:
#Eliminar columnas no informativas
del(dts['PERIODO'])
del(dts['FAMI_TIENEINTERNET.1'])
del(dts['FAMI_TIENELAVADORA'])
del(dts['FAMI_TIENEAUTOMOVIL'])
del(dts['ESTU_PRIVADO_LIBERTAD'])

In [ ]:
# Completamos datos en variables categóricas
cat_cols = dts.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    dts[col] = dts[col].fillna('Desconocido')

In [ ]:
dts.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,No,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,No,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,No,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [ ]:
def codificarCategoricas_test(X, encoders):
  X = X.copy()

  one_hot_cols = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']
  label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']
  ordinal_cols = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA']

  # Ordinal Encoder ya entrenado
  for col in ordinal_cols:
    if col in X.columns and col in encoders:
      oe = encoders[col]
      X[[col]] = oe.transform(X[[col]])

  # One-Hot
  X = pd.get_dummies(X, columns=one_hot_cols, drop_first=False)

  # Label Encoder ya entrenado
  for col in label_cols:
    if col in X.columns and col in encoders:
      le = encoders[col]
      X[col] = X[col].fillna("__desconocido__").astype(str)
      X[col] = X[col].apply(lambda x: x if x in le.classes_ else "__desconocido__")
      if "__desconocido__" not in le.classes_:
        le.classes_ = np.append(le.classes_, "__desconocido__")
      X[col] = le.transform(X[col])

  # Asegurar que las columnas coincidan con las del train
  train_cols = encoders['feature_names']

  # Agregar columnas faltantes como 0
  for col in train_cols:
    if col not in X.columns:
      X[col] = 0

  # Eliminar columnas extra
  extra_cols = set(X.columns) - set(train_cols)
  if extra_cols:
    X.drop(columns=extra_cols, inplace=True)

  # Reordenar columnas igual que en train
  X = X[train_cols]

  return X

In [ ]:
Xts = codificarCategoricas_test(dts, encoders)

In [ ]:
Xts.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,coef_1,coef_2,...,coef_4,FAMI_TIENEINTERNET_Desconocido,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,ESTU_PAGOMATRICULAPROPIO_Desconocido,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Desconocido,FAMI_TIENECOMPUTADOR_No,FAMI_TIENECOMPUTADOR_Si
0,550236,943,5,1.0,1.0,3.0,11,7,0.328,0.219,...,0.247,False,False,True,False,False,True,False,False,True
1,98545,12,1,4.0,3.0,2.0,9,11,0.227,0.283,...,0.324,False,False,True,False,True,False,False,False,True
2,499179,456,4,3.0,0.0,3.0,10,9,0.285,0.228,...,0.247,False,False,True,False,True,False,False,False,True
3,782980,229,27,3.0,3.0,1.0,8,8,0.160,0.408,...,0.294,False,True,False,False,True,False,False,True,False
4,785185,14,3,4.0,2.0,2.0,9,9,0.209,0.283,...,0.286,False,False,True,False,True,False,False,False,True


In [ ]:
print (Xtr.shape, ytr.shape)
print (Xts.shape)

(692500, 21) (692500,)
(296786, 21)


# Solución final



## Validación Cruzada

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(clf, Xtr, ytr, cv=5, scoring='accuracy')
print("Accuracies por fold:", scores)
print("Accuracy promedio:", scores.mean())

Accuracies por fold: [0.39669314 0.39376895 0.39637545 0.39635379 0.39409386]
Accuracy promedio: 0.3954570397111914


## Entrenar modelo con Random Forest y realizar predicciones para el archivo test

In [ ]:
clf.fit(Xtr, ytr)
preds_ts = clf.predict(Xts)
preds_ts_labels = oe_y.inverse_transform(preds_ts.reshape(-1, 1)).ravel()

## Generar archivo de solución para Kaggle

In [ ]:
submission = pd.DataFrame({
    'ID': dts['ID'],
    'RENDIMIENTO_GLOBAL': preds_ts_labels
})
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,medio-bajo
4,785185,bajo


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
!head submission.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-alto
499179,alto
782980,medio-bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,bajo
